# Bioclimatic variables

Compute the bioclimatic variables as defined in https://pubs.usgs.gov/ds/691/ds691.pdf. Here are the variables from the dataset to compare results: https://worldclim.org/data/worldclim21.html.

In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
from os.path import isfile, exists
from os import mkdir

In [2]:
dsspat = xr.open_zarr("https://openstack.cebitec.uni-bielefeld.de:8080/swift/v1/DWDCube/cube_spatial.zarr/")
dstime = xr.open_zarr("https://openstack.cebitec.uni-bielefeld.de:8080/swift/v1/DWDCube/cube_temporal.zarr/")
subset = dstime.isel(Time=range(0, 2*365+1)) #two year subset for faster testing

/home/giles/miniconda3/envs/challenge/lib/python3.11/site-packages/xarray/conventions.py:431: SerializationWarning: variable 'hurs' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/home/giles/miniconda3/envs/challenge/lib/python3.11/site-packages/xarray/conventions.py:431: SerializationWarning: variable 'pr' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/home/giles/miniconda3/envs/challenge/lib/python3.11/site-packages/xarray/conventions.py:431: SerializationWarning: variable 'rsds' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/home/giles/miniconda3/envs/challenge/lib/python3.11/site-packages/xarray/conventions.py:431: SerializationWarning: variable 'tas' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/home/giles/miniconda3/envs/challenge/lib/python3.11/si

In [3]:
# Lets look at the data
dstime

<xarray.Dataset>
Dimensions:                     (Time: 8766, rlat: 824, rlon: 848, vertices: 4)
Coordinates:
  * Time                        (Time) datetime64[ns] 1995-01-01 ... 2018-12-31
    height                      float64 ...
    latitude                    (rlat, rlon) float64 dask.array<chunksize=(412, 424), meta=np.ndarray>
    longitude                   (rlat, rlon) float64 dask.array<chunksize=(412, 424), meta=np.ndarray>
  * rlat                        (rlat) float64 -23.4 -23.35 ... 21.81 21.86
  * rlon                        (rlon) float64 -28.4 -28.35 ... 18.13 18.18
  * vertices                    (vertices) float64 1.0 2.0 3.0 4.0
Data variables:
    hurs                        (Time, rlat, rlon) float32 dask.array<chunksize=(1000, 103, 53), meta=np.ndarray>
    pr                          (Time, rlat, rlon) float32 dask.array<chunksize=(1000, 103, 53), meta=np.ndarray>
    rotated_latitude_longitude  float64 ...
    rsds                        (Time, rlat, rlon) float32 dask.array<chunksize=(1000, 103, 53), meta=np.ndarray>
    tas                         (Time, rlat, rlon) float32 dask.array<chunksize=(1000, 103, 53), meta=np.ndarray>
    vertices_latitude           (rlat, rlon, vertices) float64 dask.array<chunksize=(412, 424, 1), meta=np.ndarray>
    vertices_longitude          (rlat, rlon, vertices) float64 dask.array<chunksize=(412, 424, 1), meta=np.ndarray>
Attributes: (12/45)
    CDO:                            Climate Data Operators version 2.2.1 (htt...
    CORDEX_domain:                  EUR-6km
    Conventions:                    CF-1.7
    activity_id:                    COSMO-REA
    cmor_version:                   3.6.0
    contact:                        cdc.rea@dwd.de
    ...                             ...
    table_info:                     Creation Date:(27 Jan 2023) MD5:509fc5c33...
    title:                          COSMO-REA6 regional reanalysis data prepa...
    tracking_id:                    hdl:21.14103/2d8fab63-e5ec-4fef-838b-0ba9...
    variable_id:                    tas
    variant_info:                   Regional reanalysis forced by ERA-Interim.
    variant_label:                  r1i1p1f1

In [4]:
# Check for a folder to save the data
if not exists("./datasets"):
    mkdir("./datasets")

# Intermediate variables

The original definitions are based on daily maximum and minimum temperatures, which our data does not have. We only have daily mean temperatures. So there will be an inherent difference between our calculations and the data from worldclim.

### Temperature

In [5]:
# Monthly max:
if isfile("./datasets/tas_mmax.nc"):
    tas_mmax = xr.open_dataarray("./datasets/tas_mmax.nc")
else:
    tas_mmax = dstime["tas"].resample(Time="1MS").max(dim="Time")
    tas_mmax.to_netcdf("./datasets/tas_mmax.nc")
    
    
# Monthly min:
if isfile("./datasets/tas_mmin.nc"):
    tas_mmin = xr.open_dataarray("./datasets/tas_mmin.nc")
else:
    tas_mmin = dstime["tas"].resample(Time="1MS").min(dim="Time")
    tas_mmin.to_netcdf("./datasets/tas_mmin.nc")
    
    
# Monthly mean:
if isfile("./datasets/tas_mmean.nc"):
    tas_mmean = xr.open_dataarray("./datasets/tas_mmean.nc")
else:
    tas_mmean = dstime["tas"].resample(Time="1MS").mean(dim="Time")
    tas_mmean.to_netcdf("./datasets/tas_mmean.nc")

### Precipitation

In [6]:
# Monthly total:
if isfile("./datasets/prec_mtot.nc"):
    prec_mtot = xr.open_dataarray("./datasets/prec_mtot.nc")
else:
    prec_mtot = dstime["pr"].resample(Time="1MS").sum(dim="Time") * 86400 #convert from flux to total in mm
    prec_mtot = prec_mtot.assign_attrs({"standard_name": "total_precipitation", "units": "mm"})
    prec_mtot.to_netcdf("./datasets/prec_mtot.nc")

# [BIO1] Annual Mean Temperature

The annual mean temperature is defined as the average of all the 12 monthly averages.

In [7]:
ds_bio1 = tas_mmean.groupby("Time.year").mean("Time")
ds_bio1.to_netcdf("./datasets/bio1.nc")

# [BIO2] Mean Diurnal Range

The mean diurnal range is defined as mean of monthly maximum temperatures and minimum temperatures, i.e.: MDR $= \frac{1}{12} \sum_{i=1}^{12} T_\text{max} - T_\text{min}$

In [8]:
monthly_range = tas_mmax - tas_mmin

ds_bio2 = monthly_range.groupby("Time.year").mean(dim="Time")
ds_bio2.to_netcdf("./datasets/bio2.nc")

# [BIO4] Temperature Seasonality

Temperature seasonality is defined as the standard deviation (over one year) of monthly temperature averages.

In [9]:
ds_bio4 = tas_mmean.groupby("Time.year").std(dim="Time")
ds_bio4.to_netcdf("./datasets/bio4.nc")

# [BIO5] Max Temperature of Warmest Month

In [10]:
ds_bio5 = tas_mmax.groupby("Time.year").max(dim="Time")
ds_bio5.to_netcdf("./datasets/bio5.nc")

# [BIO6] Min Temperature of Coldest Month

In [11]:
ds_bio6 = tas_mmin.groupby("Time.year").min(dim="Time")
ds_bio6.to_netcdf("./datasets/bio6.nc")

# [BIO7] Temperature Annual Range

Defined as: $\text{BIO5} - \text{BIO6}$

In [12]:
ds_bio7 = ds_bio5 - ds_bio6
ds_bio7.to_netcdf("./datasets/bio7.nc")

# [BIO3] Isothermality

Isothermality quantifies how large the day-to-night temperatures oscillate relative to the summer-to-winter (annual) oscillations.
Defined as: $\frac{\text{BIO2}}{\text{BIO7}} \cdot 100$\

It does not make much sense to compute this since we dont have daily maximum and minimum temperatures, but we'll do it anyways.

Execute below cell only after computing BIO7...

In [13]:
ds_bio3 = ds_bio2 / ds_bio7 * 100.
ds_bio3.to_netcdf("./datasets/bio3.nc")

# [BIO8/9] Mean Temperature of Wettest/Driest Quarter

In our [reference](https://pubs.usgs.gov/ds/691/ds691.pdf), the quartals are defined as 12 rolling three month windows looking forward, i.e., Jan-Feb-Mar, Feb-Mar-Apr, ..., Okt-Nov-Dec, Nov-Dec-Jan, Dec-Jan-Feb (12 in total). xarrays rolling functionality only does backwards or centered rolling windows. Then, we first invert the data along the time dimension, we do the rolling sum/mean, and we invert the result.

In [14]:
quarterly_precsum = prec_mtot[::-1].rolling({"Time":3}, min_periods=1).sum()[::-1]
quarterly_tasmean = tas_mmean[::-1].rolling({"Time":3}, min_periods=1).mean()[::-1]

In [15]:
# argmax and argmin are not available for groupby so we have to apply them with a workaround
wettest_quarters = quarterly_precsum.groupby("Time.year").apply(lambda c: c.argmax(dim="Time"))
driest_quarters = quarterly_precsum.groupby("Time.year").apply(lambda c: c.argmin(dim="Time"))

In [107]:
# we need to select the corresponding index of wettest_quarters or driest_quarters for each year
# let's make a function to do that
# The function extracts the year of each group, and selects the index of the group based on the index of the corresponding year
def isel_index(group, index):
    yy = group.Time.dt.year.values[0]
    return group.isel(Time=index.sel(year=yy))

In [108]:
ds_bio8 = quarterly_tasmean.groupby("Time.year").apply(isel_index, index=wettest_quarters)
ds_bio9 = quarterly_tasmean.groupby("Time.year").apply(isel_index, index=driest_quarters)

Let's check that it worked as expected. The BIO8 for the 5th year is (at a random point 500,500):


In [114]:
ds_bio8[4,500,500]

<xarray.DataArray 'tas' ()>
array(288.55307, dtype=float32)
Coordinates:
    rlon       float64 -0.9024
    latitude   float64 54.84
    longitude  float64 16.44
    rlat       float64 4.098
    Time       datetime64[ns] 1999-08-01
    height     float64 2.0
    year       int64 1999
Attributes:
    cell_measures:  area: areacella
    cell_methods:   area: time: mean
    comment:        near-surface (usually, 2 meter) air temperature
    grid_mapping:   rotated_latitude_longitude
    history:        2023-06-30T10:58:24Z altered by CMOR: Treated scalar dime...
    long_name:      Near-Surface Air Temperature
    name:           tas
    original_name:  T2M
    standard_name:  air_temperature
    units:          K

The wettest quarter in the fifth year at that point is:

In [115]:
wettest_quarters[4, 500, 500].values

array(7)

Let's check that that is true:

In [116]:
quarterly_precsum[12*4:12*5, 500, 500]

<xarray.DataArray 'pr' (Time: 12)>
array([177.9667927 , 170.37695758, 172.6357514 , 216.2978461 ,
       219.86424197, 335.27930193, 303.62501778, 350.27734973,
       211.63475029, 290.82027934, 233.20603743, 264.95407987])
Coordinates:
    rlon       float64 -0.9024
    latitude   float64 ...
    longitude  float64 ...
    rlat       float64 4.098
  * Time       (Time) datetime64[ns] 1999-01-01 1999-02-01 ... 1999-12-01
    height     float64 ...
Attributes:
    standard_name:  total_precipitation
    units:          mm

Indeed, the value at index=7 is the maximum. Then, the mean temperature at index=7 for year 5 is:

In [117]:
quarterly_tasmean[12*4+7, 500, 500]

<xarray.DataArray 'tas' ()>
array(288.55307, dtype=float32)
Coordinates:
    rlon       float64 -0.9024
    latitude   float64 ...
    longitude  float64 ...
    rlat       float64 4.098
    Time       datetime64[ns] 1999-08-01
    height     float64 ...
Attributes:
    cell_measures:  area: areacella
    cell_methods:   area: time: mean
    comment:        near-surface (usually, 2 meter) air temperature
    grid_mapping:   rotated_latitude_longitude
    history:        2023-06-30T10:58:24Z altered by CMOR: Treated scalar dime...
    long_name:      Near-Surface Air Temperature
    name:           tas
    original_name:  T2M
    standard_name:  air_temperature
    units:          K

Which is indeed the same value as the one in BIO8.

Let's save the datasets:

In [118]:
ds_bio8.to_netcdf("./datasets/bio8.nc")
ds_bio9.to_netcdf("./datasets/bio9.nc")

# [BIO10/11] Mean Temperature of Warmest/Coldest Quarter

Analogously as before:

In [120]:
# argmax and argmin are not available for groupby so we have to apply them with a workaround
warmest_quarters = quarterly_tasmean.groupby("Time.year").apply(lambda c: c.argmax(dim="Time"))
coldest_quarters = quarterly_tasmean.groupby("Time.year").apply(lambda c: c.argmin(dim="Time"))

ds_bio10 = quarterly_tasmean.groupby("Time.year").apply(isel_index, index=warmest_quarters)
ds_bio11 = quarterly_tasmean.groupby("Time.year").apply(isel_index, index=coldest_quarters)

(the original definition is based on the sum of the monthly values of each quarter, but since the location of the maximum of the quarters sum or the quarters mean is the same, we reuse the quarters mean since we already calculated them).

In [122]:
# Save:
ds_bio10.to_netcdf("./datasets/bio10.nc")
ds_bio11.to_netcdf("./datasets/bio11.nc")

# [BIO12] Annual Precipitation

In [123]:
ds_bio12 = prec_mtot.groupby("Time.year").sum(dim="Time")
ds_bio12.to_netcdf("./datasets/bio12.nc")

# [BIO13] Precipitation of Wettest Month

In [124]:
ds_bio13 = prec_mtot.groupby("Time.year").max(dim="Time")
ds_bio13.to_netcdf("./datasets/bio13.nc")

# [BIO14] Precipitation of Driest Month

In [125]:
ds_bio14 = prec_mtot.groupby("Time.year").min(dim="Time")
ds_bio14.to_netcdf("./datasets/bio14.nc")

# [BIO15] Precipitation Seasonality

Precipitation seasonality in a given year is defined as: 
$$
\frac{\sigma(P_i)}{1 + \text{BIO12}\,/\,12} \cdot 100 \quad \text{for}\, i \in [1,12]
$$

In [126]:
ds_bio15 = prec_mtot.groupby("Time.year").std(dim="Time") / (1. + ds_bio12/12.) * 100
ds_bio15.to_netcdf("./datasets/bio15.nc")

# [BIO16/17] Precipitation of Wettest/Driest Quarter

In [127]:
# analogous to BIO8-11
ds_bio16 = quarterly_precsum.groupby("Time.year").apply(isel_index, index=wettest_quarters)
ds_bio17 = quarterly_precsum.groupby("Time.year").apply(isel_index, index=driest_quarters)

In [128]:
# Save:
ds_bio16.to_netcdf("./datasets/bio16.nc")
ds_bio17.to_netcdf("./datasets/bio17.nc")

# [BIO18/19] Precipitation of Warmest/Coldest Quarter

In [129]:
# analogous to BIO8-11
ds_bio18 = quarterly_precsum.groupby("Time.year").apply(isel_index, index=warmest_quarters)
ds_bio19 = quarterly_precsum.groupby("Time.year").apply(isel_index, index=coldest_quarters)

In [130]:
# Save:
ds_bio18.to_netcdf("./datasets/bio18.nc")
ds_bio19.to_netcdf("./datasets/bio19.nc")